In [12]:
import json
import pickle
import pandas as pd

In [13]:
def build_sensor_data(file_path):
    with open(file_path) as f:
        j = json.load(f)

    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)

    df["Date"] = pd.to_datetime(
        df["Real time clock"],
        unit="s",
        utc=True).map(lambda x: x.tz_convert("Europe/Istanbul"))
    df["Date"] = df["Date"].dt.tz_localize(None)

    sensor_indexes = sorted(df["Sensor Index"].unique())

    sensors = {}
    for i in sensor_indexes:
        sensors[i] = {}
        sensor = df[df["Sensor Index"] == i]
        heater_indexes = sorted(sensor["Heater Profile Step Index"].unique())
        for j in heater_indexes:
            sensors[i][j] = sensor[sensor["Heater Profile Step Index"] == j]

    return sensors

In [14]:
sensor_data = {
    "day1": {"mat1": None, "mat2": None},
    "day2": {"mat1": None, "mat2": None},
    "day3": {"mat1": None, "mat2": None}
}

In [15]:
for day in range(1, 4):
    sensor_data[f"day{day}"]["mat1"] = build_sensor_data(f"gunler/{day}/01.bmerawdata")
    sensor_data[f"day{day}"]["mat2"] = build_sensor_data(f"gunler/{day}/02.bmerawdata")

In [16]:
sensor_data["day3"]["mat2"][2]

{0:        Sensor Index  Sensor ID  Time Since PowerOn  Real time clock  \
 32                2  355891525               16195       1727261941   
 303               2  355891525               94274       1727262019   
 565               2  355891525              172251       1727262097   
 825               2  355891525              250277       1727262176   
 1091              2  355891525              328328       1727262254   
 ...             ...        ...                 ...              ...   
 23669             2  355891525             6958183       1727268884   
 23933             2  355891525             7036120       1727268962   
 24200             2  355891525             7114180       1727269040   
 24464             2  355891525             7192092       1727269118   
 24727             2  355891525             7270154       1727269196   
 
        Temperature    Pressure  Relative Humidity  Resistance Gassensor  \
 32       27.304951  924.568420          45.000114     

In [17]:
with open("sensor_data.pkl", "wb") as f:
    pickle.dump(sensor_data, f)